# Задачи с sql_academy



In [ ]:
# Вывести имена всех людей, которые есть в базе данных авиакомпаний
SELECT name
FROM Passenger;

In [ ]:
# Вывести названия всеx авиакомпаний
SELECT name
FROM company;

In [ ]:
# Вывести все рейсы, совершенные из Москвы
SELECT *
FROM trip
WHERE town_from = 'moscow';

In [ ]:
# Вывести имена людей, которые заканчиваются на "man"
SELECT name
FROM Passenger
WHERE name LIKE '%man';

In [ ]:
# Вывести количество рейсов, совершенных на TU-134
SELECT COUNT(*) AS count
FROM Trip
WHERE plane = 'TU-134';

In [ ]:
# Какие компании совершали перелеты на Boeing
SELECT DISTINCT Company.name
FROM Trip
    INNER JOIN Company ON Trip.company = Company.id
WHERE plane = 'Boeing';

In [ ]:
# Вывести все названия самолётов, на которых можно улететь в Москву (Moscow)
SELECT DISTINCT plane
FROM Trip
WHERE town_to = 'Moscow';

In [ ]:
# В какие города можно улететь из Парижа (Paris) и сколько времени это займёт?
SELECT town_to,
    CAST(time_in - time_out AS time) AS flight_time
FROM Trip
WHERE town_from = 'Paris';

In [ ]:
# Какие компании организуют перелеты из Владивостока (Vladivostok)?
SELECT Company.name
FROM Trip
    INNER JOIN Company ON Trip.company = Company.id
WHERE town_from = 'Vladivostok';

In [ ]:
# Вывести вылеты, совершенные с 10 ч. по 14 ч. 1 января 1900 г.
SELECT *
FROM Trip
WHERE time_out >= '1900-01-01T10:00:00.000Z'
  AND time_out <= '1900-01-01T14:00:00.000Z';

In [ ]:
# Вывести пассажиров с самым длинным именем
SELECT name
FROM Passenger
WHERE LENGTH(name) = (SELECT MAX(LENGTH(name)) 
                      FROM Passenger);

In [ ]:
# Вывести id и количество пассажиров для всех прошедших полётов
SELECT trip,
    COUNT(passenger) AS count
FROM Pass_in_trip
GROUP BY trip;

In [ ]:
# Вывести имена людей, у которых есть полный тёзка среди пассажиров
SELECT name
FROM Passenger
GROUP BY name
HAVING COUNT(name) > 1;

In [ ]:
# В какие города летал Bruce Willis
SELECT DISTINCT Trip.town_to
FROM Passenger
    INNER JOIN Pass_in_trip ON Passenger.id = Pass_in_trip.passenger
    INNER JOIN Trip ON Pass_in_trip.trip = Trip.id
WHERE Passenger.name = 'Bruce Willis';

In [ ]:
# Выведите дату и время прилёта пассажира Стив Мартин (Steve Martin) в Лондон (London)
SELECT time_in 
FROM Trip 
JOIN Pass_in_trip
    ON Trip.id = Pass_in_trip.trip
JOIN Passenger
    ON Pass_in_trip.passenger = Passenger.id
WHERE Passenger.name = 'Steve Martin'
  AND town_to = 'London';

In [ ]:
# Вывести отсортированный по количеству перелетов (по убыванию) и имени (по возрастанию) список 
# пассажиров, совершивших хотя бы 1 полет.
SELECT Passenger.name, 
       COUNT(trip) AS count
FROM Passenger
JOIN Pass_in_trip
    ON Pass_in_trip.passenger = Passenger.id
GROUP BY Passenger.name
HAVING COUNT(trip) > 0
ORDER BY count DESC, name;

In [ ]:
# Определить, сколько потратил в 2005 году каждый из членов семьи
SELECT member_name,
       status,
       SUM(unit_price * amount) AS costs
FROM FamilyMembers 
JOIN Payments
    ON FamilyMembers.member_id = Payments.family_member
WHERE EXTRACT(YEAR FROM date) = 2005
GROUP BY member_name, status;

In [ ]:
# Узнать, кто старше всех в семьe
SELECT member_name
FROM FamilyMembers
ORDER BY birthday
LIMIT 1;

SELECT member_name
FROM FamilyMembers 
WHERE birthday = (SELECT MIN(birthday) 
                  FROM FamilyMembers);

In [ ]:
# Определить, кто из членов семьи покупал картошку (potato)
SELECT DISTINCT status
FROM FamilyMembers
JOIN Payments
    ON FamilyMembers.member_id = Payments.family_member
JOIN Goods
    ON Payments.good = Goods.good_id
WHERE good_name = 'potato';

In [ ]:
# Сколько и кто из семьи потратил на развлечения (entertainment). Вывести статус в семье, имя, сумму
SELECT status, 
       member_name, 
       SUM(unit_price * amount) AS costs
FROM FamilyMembers
JOIN Payments
    ON member_id = family_member
JOIN Goods
    ON good = good_id
JOIN GoodTypes
    ON type = good_type_id
WHERE good_type_name = 'entertainment'
GROUP BY status, 
         member_name;

In [ ]:
# Определить товары, которые покупали более 1 раза
SELECT good_name
FROM Goods
JOIN Payments
    ON Goods.good_id = Payments.good
GROUP BY good_name
HAVING COUNT(*) > 1;

In [ ]:
# Найти имена всех матерей (mother)
SELECT member_name 
FROM FamilyMembers
WHERE status = 'mother';

In [ ]:
# Найдите самый дорогой деликатес (delicacies) и выведите его стоимость
SELECT good_name,
       unit_price
FROM Payments
JOIN Goods
    ON Goods.good_id = Payments.good
JOIN GoodTypes
    ON Goods.type = GoodTypes.good_type_id
WHERE good_type_name = 'delicacies'
ORDER BY unit_price DESC
LIMIT 1;

In [ ]:
# Определить кто и сколько потратил в июне 2005
SELECT member_name,
       SUM(unit_price * amount) AS costs
FROM FamilyMembers 
JOIN Payments
    ON FamilyMembers.member_id = Payments.family_member
WHERE EXTRACT(YEAR FROM date) = 2005
  AND EXTRACT(MONTH FROM date) = 6
GROUP BY member_name;

In [ ]:
# Определить, какие товары не покупались в 2005 году
SELECT good_name 
FROM Goods
WHERE good_id NOT IN (SELECT good
                      FROM Payments
                      WHERE EXTRACT(YEAR FROM date) = 2005);

In [ ]:
# Определить группы товаров, которые не приобретались в 2005 году
SELECT DISTINCT good_type_name 
FROM GoodTypes
WHERE good_type_id NOT IN (SELECT good_type_id 
                      FROM GoodTypes
                      JOIN Goods ON good_type_id = type
                      JOIN Payments ON good_id=good
                      WHERE EXTRACT(YEAR FROM date) = 2005);

In [ ]:
# Узнать, сколько потрачено на каждую из групп товаров в 2005 году. Вывести название группы и сумму
SELECT DISTINCT good_type_name,
       SUM(unit_price * amount) AS costs
FROM GoodTypes
JOIN Goods ON good_type_id = type
JOIN Payments ON good_id=good
WHERE EXTRACT(YEAR FROM date) = 2005
GROUP BY good_type_name;

In [ ]:
# Сколько рейсов совершили авиакомпании из Ростова (Rostov) в Москву (Moscow) ?
SELECT COUNT(*) AS count
FROM Trip
WHERE town_to = 'Moscow'
  AND town_from = 'Rostov';

In [ ]:
# Выведите имена пассажиров улетевших в Москву (Moscow) на самолете TU-134
SELECT DISTINCT Passenger.name 
FROM Passenger 
JOIN Pass_in_trip
    ON Passenger.id = Pass_in_trip.passenger
JOIN Trip
    ON Trip.id = Pass_in_trip.trip
WHERE town_to = 'Moscow'
  AND plane = 'TU-134';

In [ ]:
# Выведите нагруженность (число пассажиров) каждого рейса (trip). 
#Результат вывести в отсортированном виде по убыванию нагруженности.
SELECT trip,
       COUNT(*) AS count
FROM Pass_in_trip
GROUP BY trip
ORDER BY count DESC;

In [ ]:
# Вывести всех членов семьи с фамилией Quincey.
SELECT *
FROM FamilyMembers
WHERE member_name LIKE '%Quincey';

In [ ]:
# Вывести средний возраст людей (в годах), хранящихся в базе данных. Результат округлите до целого в меньшую сторону
SELECT FLOOR(AVG(EXTRACT(YEAR FROM CURRENT_DATE) - EXTRACT(YEAR FROM birthday))) AS age
FROM FamilyMembers;

In [ ]:
# Найдите среднюю стоимость икры. В базе данных хранятся данные о покупках красной (red caviar) и черной икры (black caviar).
SELECT AVG(unit_price) AS cost
FROM Payments
JOIN Goods
    ON Payments.good = Goods.good_id
WHERE good_name IN ('red caviar', 'black caviar');

In [ ]:
# Сколько всего 10-ых классов
SELECT COUNT(*) as count
FROM Class
WHERE name LIKE '10%';

In [ ]:
# Сколько различных кабинетов школы использовались 2.09.2019 в образовательных целях?
SELECT COUNT(*) AS count
FROM Schedule
WHERE date = '2019-09-02';

In [ ]:
# Выведите информацию об обучающихся живущих на улице Пушкина (ul. Pushkina)?
SELECT *
FROM Student
WHERE address LIKE 'ul. Pushkina%';

In [ ]:
# Сколько лет самому молодому обучающемуся?
SELECT MIN(TIMESTAMPDIFF(YEAR, birthday, CURRENT_DATE)) AS year
FROM Student;

In [ ]:
# Сколько Анн (Anna) учится в школе ?
SELECT COUNT(first_name) AS count
FROM Student
WHERE first_name = 'Anna';

In [ ]:
# Сколько обучающихся в 10 B классе ?
SELECT COUNT(student) AS count
FROM Student_in_class
JOIN Class
    ON Student_in_class.class = Class.id
WHERE Class.name = '10 B';

In [ ]:
# Выведите название предметов, которые преподает Ромашкин П.П. (Romashkin P.P.) ?
SELECT name AS subjects
FROM Subject
JOIN Schedule 
    ON Subject.id=Schedule.subject
JOIN Teacher 
    ON Schedule.teacher = Teacher.id
WHERE Teacher.last_name='Romashkin'
  AND Teacher.first_name LIKE 'P%'
  AND Teacher.middle_name LIKE 'P%';

In [ ]:
# Во сколько начинается 4-ый учебный предмет по расписанию ?
SELECT start_pair
FROM Timepair
WHERE id = 4;

In [ ]:
# Сколько времени обучающийся будет находиться в школе, учась со 2-го по 4-ый уч. предмет ?
SELECT DISTINCT TIMEDIFF( 
                (SELECT end_pair FROM Timepair WHERE id = 4),
                (SELECT start_pair FROM Timepair WHERE id = 2)
                ) AS time
FROM Timepair;

In [ ]:
# Выведите фамилии преподавателей, которые ведут физическую культуру (Physical Culture). Отcортируйте преподавателей по фамилии.
SELECT last_name
FROM Teacher
JOIN Schedule
    ON Teacher.id=Schedule.teacher
JOIN Subject
    ON Schedule.subject = Subject.id
WHERE Subject.name = 'Physical Culture'
ORDER BY Teacher.last_name;

In [ ]:
# Найдите максимальный возраст (колич. лет) среди обучающихся 10 классов ?
SELECT MAX(TIMESTAMPDIFF(YEAR, birthday, CURRENT_DATE)) AS max_year
FROM Student
JOIN Student_in_class
    ON Student.id = Student_in_class.student
JOIN Class 
    ON Student_in_class.class = Class.id
WHERE Class.name LIKE '10%';

In [ ]:
# Какой(ие) кабинет(ы) пользуются самым большим спросом?
SELECT classroom 
FROM Schedule
GROUP BY classroom
HAVING COUNT(classroom) = (SELECT COUNT(classroom) 
                           FROM Schedule 
                           GROUP BY classroom
                           ORDER BY COUNT(classroom) DESC 
                           LIMIT 1);

In [ ]:
# В каких классах введет занятия преподаватель "Krauze" ?
SELECT DISTINCT name
FROM Class
JOIN Schedule
    ON Class.id = Schedule.class
JOIN Teacher
    ON Schedule.teacher = Teacher.id
WHERE Teacher.last_name = 'Krauze';

In [ ]:
# Сколько занятий провел Krauze 30 августа 2019 г.?
SELECT COUNT(teacher) AS count
FROM Schedule
WHERE date = '2019-08-30'and teacher = (SELECT id 
                                       FROM Teacher
                                       WHERE last_name = 'Krauze');

In [ ]:
# Выведите заполненность классов в порядке убывания
SELECT Class.name, 
       COUNT(Student_in_class.student) AS count
FROM Class
JOIN Student_in_class
    ON Class.id = Student_in_class.class
GROUP BY Class.id
ORDER BY count DESC;

In [ ]:
# Какой процент обучающихся учится в 10 A классе ?
SELECT COUNT(student) * 100 / (SELECT COUNT(student) FROM Student_in_class) AS percent
FROM Student_in_class
JOIN Class 
    ON Student_in_class.class = Class.id
WHERE name='10 A';

In [ ]:
# Какой процент обучающихся родился в 2000 году? Результат округлить до целого в меньшую сторону.
SELECT FLOOR(COUNT(id) * 100 / (SELECT COUNT(id) FROM Student)) AS percent
FROM Student
WHERE EXTRACT(YEAR FROM birthday) = 2000;